# Exploration of the ridb.recreation.gov API

Author: Stephan <stephan@bayesimpact.org>

The API is pretty well documented on https://usda.github.io/RIDB/ and can be used to obtain information about *Recreation Areas*, *Facilities*, *Campsites*, *Tours*, etc..

In order to get an overview of the functionality of this API, I'll use it to plan a trip to the [Zion National Park](https://www.nps.gov/zion/index.htm).

In [1]:
import folium
from IPython.display import Image, HTML, display
import requests
from pprint import pprint

# Register for API key here: https://ridb.recreation.gov/?action=register
API_KEY = '6A7F1BDCC2E94496BAAAEBCDC8B17CE3'

BASE_URL = 'https://ridb.recreation.gov/api/v1/'

Reading the documentation I learned that some endpoints have a `query` parameter that can be used to filter the results. This parameter behaves differently on each endpoint. On the `recareas` endpoint it

> Searches on recreation area name, description, keywords, and stay limit.

In [2]:
def get_as_json(url, params={'apikey': API_KEY}):
    """Helper function to take care of authentication via API key."""
    res = requests.get(url, params=params)
    return res.json()['RECDATA']

In [3]:
url = BASE_URL + 'recareas'
params = {'apikey': API_KEY, 'query': 'zion'}
zion_areas = get_as_json(url, params=params)

Display the names and coordinates of the search results.

In [4]:
[(area['RecAreaName'], area.get('GEOJSON')) for area in zion_areas]

[(u'Zion National Park',
  {u'COORDINATES': [-112.988339067, 37.2020727396], u'TYPE': u'Point'}),
 (u'Zion Wilderness', None),
 (u'Beaver Canyon Scenic Byway (U-153)', None),
 (u'Smithsonian Butte National Back Country Byway', None)]

The first one looks like what I'm interested in. It's also the only one that has coordinates.

In [5]:
zion_park = zion_areas[0]
zion_park['RecAreaDescription']

u"Follow the paths where ancient native people and pioneers walked. Gaze up at massive sandstone cliffs of cream, pink, and red that soar into a brilliant blue sky. Experience wilderness in a narrow slot canyon. Zion's unique array of plants and animals will enchant you as you absorb the rich history of the past and enjoy the excitement of present day adventures."

Oh that sounds beautiful, where exactly is it?

In [6]:
import folium
zion_park_coordinates = zion_park['GEOJSON']['COORDINATES'][::-1]
map_osm = folium.Map(location=zion_park_coordinates, 
                     zoom_start=5, tiles='Stamen Terrain')
folium.RegularPolygonMarker(zion_park_coordinates).add_to(map_osm)
map_osm

Who in the world picked that marker color?

Ok, what else do I know about this park?

In [7]:
zion_park.keys()

[u'OrgRecAreaID',
 u'LastUpdatedDate',
 u'RecAreaDirections',
 u'StayLimit',
 u'RecAreaFeeDescription',
 u'RecAreaName',
 u'GEOJSON',
 u'RecAreaEmail',
 u'RecAreaReservationURL',
 u'RecAreaLongitude',
 u'RecAreaLatitude',
 u'RecAreaDescription',
 u'Keywords',
 u'RecAreaMapURL',
 u'RecAreaID',
 u'RecAreaPhone']

Let's check out the facilities of this park.

In [8]:
url = BASE_URL + '/recareas/%s/facilities/' % zion_park['RecAreaID']
zion_facilities = get_as_json(url)
[fac[u'FacilityName'] for fac in zion_facilities]

[u'WATCHMAN CAMPGROUND', u'South Campground']

Both of them seem to be campgrounds, I will display them on the map.

In [9]:
map_osm = folium.Map(location=zion_park_coordinates, 
                     zoom_start=13, tiles='Stamen Terrain')
folium.RegularPolygonMarker(zion_park_coordinates).add_to(map_osm)
for fac in zion_facilities:
    coordinates = fac['GEOJSON']['COORDINATES'][::-1]
    folium.RegularPolygonMarker(
        coordinates,
        popup=fac[u'FacilityDescription'],
        fill_color='#aa9d96',
        number_of_sides=8,
        radius=10
        ).add_to(map_osm)
map_osm.zoom_start
map_osm

Click on the campgrounds for their descriptions.

What will the media endpoint for the zion area give us?

In [10]:
url = BASE_URL + 'recareas/%s/media/' % zion_park['RecAreaID']
media = get_as_json(url)

In [11]:
image_tag = """
<div style='display: inline-block;'>
    <img style='width: 200px;' src='{URL}' />
    <span>{Title}</span>
</div>
"""

imagesList=''.join([image_tag.format(**m) for m in media])
display(HTML(imagesList))

This *South Campground* looks cool! What will the `campsites` endpoint tell me about it?

In [12]:
south_campground_fac = zion_facilities[1]
url = BASE_URL + 'facilities/%s/campsites' % south_campground_fac['FacilityID']
campsites = get_as_json(url)
pprint(campsites)

[]


Oh, that sucks. I liked that one. Ok ok, will check out the other one.

In [13]:
other_campground_fac = zion_facilities[0]
url = BASE_URL + 'facilities/%s/campsites' % other_campground_fac['FacilityID']
campsites = get_as_json(url)
print('There are %d campsites for the %s campground \n' % (len(campsites), other_campground_fac['FacilityName']))
print('Information for campsite 1:')
pprint(campsites[0])

There are 50 campsites for the WATCHMAN CAMPGROUND campground 

Information for campsite 1:
{u'ATTRIBUTES': [{u'AttributeID': 93109960,
                  u'AttributeName': u'Checkin Time',
                  u'AttributeValue': u'12:00 PM'},
                 {u'AttributeID': 93109961,
                  u'AttributeName': u'Checkout Time',
                  u'AttributeValue': u'11:00 AM'},
                 {u'AttributeID': 93109962,
                  u'AttributeName': u'Fire Pit',
                  u'AttributeValue': u'Y'},
                 {u'AttributeID': 93109963,
                  u'AttributeName': u'Max Num of People',
                  u'AttributeValue': 25.0},
                 {u'AttributeID': 93109964,
                  u'AttributeName': u'Max Num of Vehicles',
                  u'AttributeValue': 6.0},
                 {u'AttributeID': 93109965,
                  u'AttributeName': u'Min Num of People',
                  u'AttributeValue': 15.0},
                 {u'AttributeID': 9

Ah wow, that's really one individual Site on a Campground. Pretty cool.

So, are there any events going on while I'm there?

In [14]:
url = BASE_URL + 'recareas/%s/events/' % zion_park['RecAreaID']
events = get_as_json(url)
events

[]

Booooooooooring!

In [15]:
url = BASE_URL + 'recareas/%s/activities/' % zion_park['RecAreaID']
activities = get_as_json(url)
[a[u'ActivityName'] for a in  activities]

[u'AUTO TOURING',
 u'BIKING',
 u'CLIMBING',
 u'CAMPING',
 u'FISHING',
 u'HIKING',
 u'HORSEBACK RIDING',
 u'WILDLIFE VIEWING',
 u'PHOTOGRAPHY']

Also not bad.

#### Then end

There are a few more endpoints that could be interesting. Especially `Tours` and `Permit entrances`, but I will stop here for now